In [ ]:
%matplotlib ipympl

import planning.probabilistic_road_map_python as prmp
import experimental.beacon_sim.beacon_potential_python as bpp
from experimental.beacon_sim.mapped_landmarks_pb2 import MappedLandmarks
import experimental.beacon_sim.correlated_beacons_python as cbp
import experimental.beacon_sim.ekf_slam_python as esp
from common.math.matrix_pb2 import Matrix as MatrixProto
import common.time.robot_time_python as rtp
import common.liegroups.se2_python as se2

import experimental.beacon_sim.belief_road_map_planner_python as brmp

import matplotlib.pyplot as plt
import matplotlib as mpl

import numpy as np
import itertools
import tempfile

In [ ]:
def plot_mapped_landmarks(mapped_landmarks):
    ids = mapped_landmarks.beacon_ids
    xs = []
    ys = []
    for i, beacon_in_local in enumerate(mapped_landmarks.beacon_in_local):
        beacon_in_map = mapped_landmarks.beacon_in_local[i]
        xs.append(beacon_in_map.data[0])
        ys.append(beacon_in_map.data[1])
        plt.text(beacon_in_map.data[0], beacon_in_map.data[1], f't_{ids[i]}')
    plt.plot(xs, ys, 'o')

In [ ]:

def plot_road_map(road_map):
    pts = np.stack(road_map.points())
    plt.plot(pts[:, 0], pts[:, 1], '*')
    for i, pt in enumerate(pts):
        plt.text(*pt, str(i))

    if road_map.has_start_goal():
        for idx, marker in [(road_map.START_IDX, 'rs'), (road_map.GOAL_IDX, 'y*')]:
            pt = road_map.point(idx)
            plt.plot(*pt, marker)
            

    line_segments = []
    for i in range(len(pts)):
        for j in range(i+1, len(pts)):
            if road_map.adj()[i, j] != 0:
                line_segments.append([pts[i, :], pts[j,:]])

    if road_map.has_start_goal():
        for idx in [road_map.START_IDX, road_map.GOAL_IDX]:
            pt = road_map.point(idx)
            neighbors = road_map.neighbors(idx)
            for _, neighbor_pt in neighbors:
                line_segments.append([pt, neighbor_pt])
    

    edges = mpl.collections.LineCollection(line_segments, colors=(0.6, 0.8, 0.6, 1.0))
    ax = plt.gca()
    ax.add_collection(edges)

In [ ]:
min_x, min_y = 10, 10
step_x, step_y = 20, 20
num_columns, num_rows = 4, 4
num_pts = num_columns * num_rows

In [ ]:
# Create the road map

grid_points = []
adj = np.zeros((num_pts, num_pts))
for r, c in itertools.product(range(num_rows), range(num_columns)):
    grid_points.append((c * step_x + min_x, r * step_y + min_y))

    id = r * num_columns + c
    if c < num_columns - 1:
        neighbor_id = r * num_columns + c + 1
        adj[id, neighbor_id] = 1
        adj[neighbor_id, id] = 1

    if r < num_rows - 1:
        neighbor_id = (r+1) * num_columns + c
        adj[id, neighbor_id] = 1
        adj[neighbor_id, id] = 1

    if c < num_columns - 1 and r < num_rows - 1:
        neighbor_id = (r+1) * num_columns + c + 1
        adj[id, neighbor_id] = 1
        adj[neighbor_id, id] = 1

    if c > 0 and r < num_rows - 1:
        neighbor_id = (r+1) * num_columns + c - 1
        adj[id, neighbor_id] = 1
        adj[neighbor_id, id] = 1
        

start = (5, 75)
goal = (75, 5)
CONNECTION_RADIUS_M = 10

start_goal = prmp.StartGoalPair(np.array([start]).transpose(), np.array([goal]).transpose(), CONNECTION_RADIUS_M)

road_map = prmp.RoadMap(grid_points, adj, start_goal)


In [ ]:
# Create the beacons
GRID_BEACON_OFFSET = np.array((1, 1))

map_proto = MappedLandmarks()
for i, pt in enumerate(road_map.points()):
    map_proto.beacon_ids.append(i + 200)
    beacon_in_local = pt + GRID_BEACON_OFFSET
    beacon_in_local_proto = MatrixProto(
        num_rows=2,
        num_cols=1,
        data = beacon_in_local.tolist()
    )
    map_proto.beacon_in_local.append(beacon_in_local_proto)

START_GOAL_OFFSETS = [(2, 2),(-2, -2),(2, -2)]
for idx in [road_map.START_IDX, road_map.GOAL_IDX]:
    for offset in START_GOAL_OFFSETS:
        map_proto.beacon_ids.append(map_proto.beacon_ids[-1] + 1)
        beacon_in_local = road_map.point(idx) + offset
        beacon_in_local_proto = MatrixProto(
            num_rows=2,
            num_cols=1,
            data = beacon_in_local.tolist()
        )
        map_proto.beacon_in_local.append(beacon_in_local_proto)
    
    ...

num_beacons = len(map_proto.beacon_ids)
cov_in_local = np.eye(num_beacons * 2) * 0.1
cov_in_local = MatrixProto(
    num_rows = num_beacons * 2,
    num_cols = num_beacons * 2,
    data = list(itertools.chain(*cov_in_local.tolist()))
)
map_proto.cov_in_local.CopyFrom(cov_in_local)

In [ ]:
# Reliable corner
def define_potential(p_corner: float):
    corner_beacon = [200]
    anticorrelated_beacons = [205, 210]
    start_beacons = [216, 217, 218]
    absent_beacons = set(map_proto.beacon_ids) - set(corner_beacon) - set(anticorrelated_beacons) - set(start_beacons)
    
    eps = 1e-9
    start_pot= bpp.BeaconPotential.correlated_beacon_potential(
        p_present = 1.0 - eps,
        p_beacon_given_present = 1.0 - eps,
        members = start_beacons
    )
    
    corner_pot = bpp.BeaconPotential.correlated_beacon_potential(
        p_present = 1.0 - eps,
        p_beacon_given_present = p_corner,
        members = corner_beacon 
    )
    
    anticorrelated_clique = cbp.BeaconClique(0.5 - eps, eps, anticorrelated_beacons)
    anticorrelated_potential = cbp.create_correlated_beacons(anticorrelated_clique)
    
    absent_potential = bpp.BeaconPotential.correlated_beacon_potential(
        p_present = eps,
        p_beacon_given_present = eps,
        members = list(absent_beacons)
    )
    
    beacon_potential = bpp.BeaconPotential.combined_potential([start_pot, corner_pot, anticorrelated_potential, absent_potential])
    return beacon_potential



In [ ]:
eps = 1e-9
beacon_potential = define_potential(1-eps)
beacon_potential = define_potential(0.8)
beacon_potential = define_potential(eps)


In [ ]:

plt.figure()
plot_road_map(road_map)
plot_mapped_landmarks(map_proto)

plt.axis('equal')


In [ ]:
ekf_config = esp.EkfSlamConfig(
    max_num_beacons=40,
    initial_beacon_uncertainty_m=100.0,
    along_track_process_noise_m_per_rt_meter=0.02,
    cross_track_process_noise_m_per_rt_meter=0.02,
    pos_process_noise_m_per_rt_s=0.0001,
    heading_process_noise_rad_per_rt_meter=0.0001,
    heading_process_noise_rad_per_rt_s=0.00001,
    beacon_pos_process_noise_m_per_rt_s=0.001,
    range_measurement_noise_m=0.1,
    bearing_measurement_noise_rad=0.02,
    on_map_load_position_uncertainty_m=10.0,
    on_map_load_heading_uncertainty_rad=0.4,
)
ekf = esp.EkfSlam(ekf_config, rtp.RobotTimestamp())
with tempfile.NamedTemporaryFile() as file:
    file.write(map_proto.SerializeToString())
    ekf.load_map(file.name)

ekf.estimate.local_from_robot(se2.SE2(road_map.point(road_map.START_IDX)))

In [ ]:
def config_to_assignment(config, members):
    out = {}
    for i, setting in enumerate(config):
        if setting != '?':
            out[members[i]] = int(setting)
    return out
        
        

In [ ]:
# Expected Landmark

MAX_SENSOR_RANGE_M = 3.0
UNCERTAINTY_SIZE_OPTIONS = brmp.LandmarkBeliefRoadMapOptions.ExpectedDeterminant()
SAMPLED_BELIEF_OPTIONS = brmp.LandmarkBeliefRoadMapOptions.SampledBeliefOptions(
    max_num_components=256,
    seed = 1,
)

landmark_options = brmp.LandmarkBeliefRoadMapOptions(
    MAX_SENSOR_RANGE_M,
    UNCERTAINTY_SIZE_OPTIONS,
    SAMPLED_BELIEF_OPTIONS,
    timeout=None
)

plan = brmp.compute_landmark_belief_road_map_plan(road_map, ekf, beacon_potential, landmark_options)
print(plan.nodes)
cum_prob = 0.0
for config, config_belief in sorted(plan.beliefs[-1].belief_from_config.items(), key= lambda x: np.linalg.det(x[1].cov_in_robot)):
    if np.exp(config_belief.log_config_prob) > 0.01:
        cum_prob += np.exp(config_belief.log_config_prob)
        print(config_to_assignment(config, beacon_potential.members()), cum_prob, np.exp(config_belief.log_config_prob), np.sqrt(np.linalg.det(config_belief.cov_in_robot[:2, :2])))

In [ ]:
# Expected BRM
MAX_SENSOR_RANGE_M = 3.0

BRM_OPTIONS = brmp.BeliefRoadMapOptions(
    max_sensor_range_m = MAX_SENSOR_RANGE_M,
    uncertainty_tolerance = None,
    max_num_edge_transforms = 1000,
    timeout = None
)
NUM_WORLD_SAMPLES = 256
SEED = 0
EXPECTED_BRM_OPTIONS = brmp.ExpectedBeliefRoadMapOptions(
    NUM_WORLD_SAMPLES, SEED, BRM_OPTIONS 
)

plan = brmp.compute_expected_belief_road_map_plan(road_map, ekf, beacon_potential, EXPECTED_BRM_OPTIONS)
print(plan.nodes)
